# Домашнее задание № 4. Языковые модели

## Задание 1 (8 баллов).

В семинаре для генерации мы использовали предположение маркова и считали, что слово зависит только от 1 предыдущего слова. Но ничто нам не мешает попробовать увеличить размер окна и учитывать два или даже три прошлых слова. Для них мы еще сможем собрать достаточно статистик и, логично предположить, что качество сгенерированного текста должно вырасти.

Попробуйте сделать языковую модель, которая будет учитывать два предыдущих слова при генерации текста.
Сгенерируйте несколько текстов (3-5) и расчитайте перплексию получившейся модели.
Можно использовать данные из семинара или любые другие (можно брать только часть текста, если считается слишком долго). Перплексию рассчитывайте на 10-50 отложенных предложениях (они не должны использоваться при сборе статистик).


Подсказки:  
    - нужно будет добавить еще один тэг \<start>  
    - можете использовать тот же подход с матрицей вероятностей, но по строкам хронить биграмы, а по колонкам униграммы
    - тексты должны быть очень похожи на нормальные (если у вас получается рандомная каша, вы что-то делаете не так)
    - у вас будут словари с индексами биграммов и униграммов, не перепутайте их при переводе индекса в слово - словарь биграммов будет больше словаря униграммов и все индексы из униграммного словаря будут формально подходить для словаря биграммов (не будет ошибки при id2bigram[unigram_id]), но маппинг при этом будет совершенно неправильным

In [1]:
!pip install razdel

In [2]:
from string import punctuation
from razdel import sentenize
from razdel import tokenize as razdel_tokenize
import numpy as np
from IPython.display import Image
from IPython.core.display import HTML
from collections import Counter
import nltk
nltk.download('punkt_tab')
from nltk.tokenize import sent_tokenize
from scipy.sparse import lil_matrix, csr_matrix, csc_matrix

[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt_tab.zip.


In [4]:
# corpus = open('2ch_corpus.txt').read()

In [49]:
corpus = open('lenta.txt').read()

In [50]:
# удаляем пунктуацию + приводим к нижнему регистру

def normalize(text):
    normalized_text = [word.text.strip(punctuation) for word \
                                                            in razdel_tokenize(text)]
    normalized_text = [word.lower() for word in normalized_text if word and len(word) < 20 ]
    return normalized_text

In [51]:
# получаем n-граммы

def ngrammer(tokens, n=2):
    ngrams = []
    for i in range(0,len(tokens)-n+1):
        ngrams.append(' '.join(tokens[i:i+n]))
    return ngrams

In [52]:
# получаем предложения из корпуса, в которых удалена пунктуация и слова приведены к нижнему регистру

# еще у нас есть служебные токены, которые сигнализируют начало и конец предложения

# <start> 2 раза, чтобы у нас появлялись триграммы (когда предложение начинается, у него нет двух предыдущих слов)

sentences_corpus = [['<start>', '<start>'] + normalize(text) + ['<end>'] for text in sent_tokenize(corpus[:5000000])]

In [53]:
# Разделяем данные: 90% для обучения, 10% для проверки
training_sentences = sentences_corpus[:int(0.9 * len(sentences_corpus))]
held_out_sentences = sentences_corpus[int(0.9 * len(sentences_corpus)):]

In [54]:
unigrams_corpus = Counter()
bigrams_corpus = Counter()
trigrams_corpus = Counter()

for sentence in training_sentences:
    unigrams_corpus.update(sentence)
    bigrams_corpus.update(ngrammer(sentence, n=2))
    trigrams_corpus.update(ngrammer(sentence, n=3))

In [55]:
# матрица слова на слова (инициализируем нулями)
matrix_corpus = lil_matrix((len(bigrams_corpus),
                          len(unigrams_corpus)))

# маппинг для слов (id-слово и слово-id)
# к матрице нужно обращаться по индексам
# поэтому зафиксируем порядок слов в словаре и сделаем маппинг id-слово и слово-id
id2word_corpus = list(unigrams_corpus)
word2id_corpus = {word:i for i, word in enumerate(id2word_corpus)}

# маппинг для биграмм:
bigram_keys = list(bigrams_corpus.keys())
id2bigram_corpus = {i: bigram for i, bigram in enumerate(bigram_keys)}
bigram2id_corpus = {bigram: i for i, bigram in enumerate(bigram_keys)}



# Заполняем матрицу вероятностей для триграмм
for trigram in trigrams_corpus:
    word1, word2, word3 = trigram.split()
    bigram = f"{word1} {word2}"
    # На пересечение биграммы и слова ставим вероятность встретить третье слово после биграммы
    matrix_corpus[bigram2id_corpus[bigram], word2id_corpus[word3]] = (
        trigrams_corpus[trigram] / bigrams_corpus[bigram]
    )


matrix_corpus = csc_matrix(matrix_corpus)

In [56]:
def generate(matrix, id2word, word2id, id2bigram, bigram2id, n=100, start='<start>'):
    text = [start, start] # начало текста с двух токенов <start>
    current_idx = bigram2id[f"{start} {start}"] # индекс начальной биграммы

    for _ in range(n):

        chosen = np.random.choice(matrix.shape[1], p=matrix[current_idx].toarray()[0])

        next_word = id2word[chosen]
        text.append(next_word)

        if next_word == '<end>':
            text.extend([start, start])
            current_idx = bigram2id[f"{start} {start}"]
        else:
            bigram = f"{text[-2]} {text[-1]}"
            current_idx = bigram2id.get(bigram, bigram2id[f"{start} {start}"])

    text = ' '.join(text)
    text = text.replace('<start> <start>', '')
    text = text.replace('<end>', '\n')
    return text

In [57]:
for i in range(1, 6):
    print('Генерация ' + str(i) + ':')
    print(generate(matrix_corpus, id2word_corpus, word2id_corpus, id2bigram_corpus, bigram2id_corpus).replace('<end>', '\n'))
    print()

Генерация 1:
 до сих пор регистрация доменных имен в собственные руки и в машине 
  только одно из произведенных в великобритании самым теряемым предметом считался зонтик это даже было своего рода официальным аналитиком кремля и татьяны дьяченко и валерия окулова 
  благодарю бога что остался жив 
  является доцентом кафедры статистики имеет ученую степень доктора экономических наук 
  лицензии на осуществление банковских операций по счетам орт уже с самого начала 
  вывод телеведущего в ближайшие дни подать апелляцию по этому вопросу 
  кроме того масхадов заявил что в документах указанные погрешности и спустя 10 лет и старше чуть болеше обеспокоены перспективами в

Генерация 2:
 уровень процентных ставок со сроком 5 месяцев до 1 июля до 18 часов 
  в результате была пресечена деятельность 127 опг задержаны пять воров в законе 
  убийца был одет в черную куртку черные джинсы и вязанную черную шапку надвинутую на глаза похитителям когда искали работу 
  как объяснили тогда районные вла

Перплексия

In [58]:
# Функция для расчёта перплексии
def perplexity(logp, N):
    return np.exp((-1 / N) * logp)

In [59]:
def compute_join_proba_markov_assumption(text, bigram_counts, trigram_counts):
    prob = 0
    tokens = normalize(text)
    tokens = ['<start>', '<start>'] + tokens + ['<end>']  # Добавляем маркеры начала и конца
    for trigram in ngrammer(tokens, n=3):
        word1, word2, word3 = trigram.split()
        bigram = f"{word1} {word2}"
        if bigram in bigram_counts and trigram in trigram_counts:
            # Условная вероятность P(w3 | w1, w2)
            prob += np.log(trigram_counts[trigram] / bigram_counts[bigram])
        else:
            # Сглаживание для отсутствующих триграмм
            prob += np.log(2e-5)
    return prob, len(tokens) - 3  # Возвращаем лог-вероятность и количество слов (без <start> <start>)

In [60]:
ps = []
for sent in held_out_sentences:
    # Склеиваем токены в строку, если sent — это список
    if isinstance(sent, list):
        sent = ' '.join(sent)

    prob, N = compute_join_proba_markov_assumption(sent, bigrams_corpus, trigrams_corpus)
    if not N:  # Пропускаем пустые предложения
        continue
    ps.append(perplexity(prob, N))

In [61]:
# Средняя перплексия
np.mean(ps)

50460.61994295969

## Задание № 2* (2 балла).

Измените функцию generate_with_beam_search так, чтобы она работала с моделью, которая учитывает два предыдущих слова.
Сравните получаемый результат с первым заданием.
Также попробуйте начинать генерацию не с нуля (подавая \<start> \<start>), а с какого-то промпта. Но помните, что учитываться будут только два последних слова, так что не делайте длинные промпты.

In [62]:
# сделаем класс чтобы хранить каждый из лучей
class Beam:
    def __init__(self, sequence: list, score: float):
        self.sequence: list = sequence
        self.score: float = score

In [63]:
def generate_with_beam_search(matrix, id2word, word2id, id2bigram, bigram2id, n=100, max_beams=5, start='<start> <start>', prompt=None):
    # Начальное состояние
    if prompt:
        # Если задан промпт, преобразуем его в список слов
        initial_sequence = normalize(prompt)
        if len(initial_sequence) < 2:
            initial_sequence = ['<start>'] + initial_sequence
        else:
            initial_sequence = initial_sequence[-2:]  # Берём только два последних слова
    else:
        initial_sequence = ['<start>', '<start>']

    initial_node = Beam(sequence=initial_sequence, score=0.0)
    beams = [initial_node]

    for _ in range(n):
        # Список новых лучей
        new_beams = []
        for beam in beams:
            # Если последовательность уже закончена, оставляем её без изменений
            if beam.sequence[-1] == '<end>':
                new_beams.append(beam)
                continue

            # Получаем последние два слова из последовательности
            if len(beam.sequence) < 2:
                bigram = '<start> <start>'
            else:
                bigram = f"{beam.sequence[-2]} {beam.sequence[-1]}"

            # Проверяем наличие биграммы
            if bigram not in bigram2id:
                continue

            # Получаем индекс биграммы
            bigram_idx = bigram2id[bigram]

            # Вероятности продолжений
            probas = matrix[bigram_idx].toarray()[0]

            # Выбираем топ самых вероятных продолжений
            top_idxs = probas.argsort()[:-(max_beams+1):-1]

            for top_id in top_idxs:
                # Пропускаем нулевые вероятности
                if not probas[top_id]:
                    continue

                # Создаём новый луч
                new_sequence = beam.sequence + [id2word[top_id]]
                new_score = beam.score + np.log(probas[top_id])
                new_beam = Beam(sequence=new_sequence, score=new_score)
                new_beams.append(new_beam)

        # Сортируем и берём только `max_beams` лучших лучей
        beams = sorted(new_beams, key=lambda x: x.score, reverse=True)[:max_beams]

    # Возвращаем отсортированные последовательности по вероятности
    sorted_sequences = sorted(beams, key=lambda x: x.score, reverse=True)
    sorted_sequences = [" ".join(beam.sequence).replace('<start>', '').strip() for beam in sorted_sequences]
    return sorted_sequences

In [64]:
# Генерация с началом по умолчанию
generated_sequences = generate_with_beam_search(
    matrix_corpus, id2word_corpus, word2id_corpus, id2bigram_corpus, bigram2id_corpus,
    n=50, max_beams=5
)
print("Сгенерированные последовательности (с <start>):")
for seq in generated_sequences:
    print(seq)

Сгенерированные последовательности (с <start>):
как сообщает риа новости <end>
как сообщает агентство риа новости <end>
как сообщает риа новости со ссылкой на источники в правоохранительных органах грузии <end>
как сообщает риа новости со ссылкой на источники в правоохранительных органах столицы <end>
как сообщает риа новости со ссылкой на источники в правоохранительных органах города интерфаксу сообщили в пресс-службе президента армении <end>


In [65]:
# Генерация с промптом
prompt = "на днях"
generated_sequences_with_prompt = generate_with_beam_search(
    matrix_corpus, id2word_corpus, word2id_corpus, id2bigram_corpus, bigram2id_corpus,
    n=50, max_beams=5, prompt=prompt
)
print("Сгенерированные последовательности (с промптом):")
for seq in generated_sequences_with_prompt:
    print(seq)

Сгенерированные последовательности (с промптом):
на днях в железногорске состоялось торжественное открытие международного делового центра <end>
на днях в железногорске состоялось торжественное открытие международного делового центра москва-сити <end>
на днях в селении дучи новолакского района дагестана <end>
на днях в селении верхний наур было убито 170 человек <end>
на днях в селении верхний наур было убито и шестеро солдат <end>
